# Figure 1
## Demonstrating pEYE's Visualizations - Single Trial Level

In [8]:
import os

import cv2
import numpy as np
import pandas as pd
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.io as pio

import peyes

import analysis.utils as u
from analysis._article_results.lund2013._helpers import *

pio.renderers.default = "browser"

### Load Data
**(1)** Stimulus

In [9]:
STIMULUS_TO_SHOW = 'konijntjes'
PATH_TO_STIMULI = os.path.join(u.BASE_DIR, "stimuli", DATASET_NAME.capitalize(), STIMULUS_TYPE)

img = cv2.imread(os.path.join(PATH_TO_STIMULI, f"{STIMULUS_TO_SHOW}.png"))
resolution = (img.shape[1], img.shape[0])

**(2)** Gaze Data

In [10]:
dataset = u.load_dataset(DATASET_NAME, verbose=False)
image_dataset = dataset[dataset[peyes.constants.STIMULUS_TYPE_STR] == peyes.constants.IMAGE_STR]

# extract single-trial data
relevant_trial_ids = (
        ~image_dataset.groupby(peyes.constants.TRIAL_ID_STR)['RA'].apply(lambda trl: trl.isnull().all()) &
        ~image_dataset.groupby(peyes.constants.TRIAL_ID_STR)['MN'].apply(lambda trl: trl.isnull().all()) &
        image_dataset.groupby(peyes.constants.TRIAL_ID_STR)[peyes.constants.STIMULUS_NAME_STR].apply(lambda trl: trl.iloc[0] == STIMULUS_TO_SHOW)
)
trial_id = relevant_trial_ids[relevant_trial_ids].index[0]
trial_data = image_dataset[image_dataset[peyes.constants.TRIAL_ID_STR] == trial_id]

# extract gaze data from single-trial
pixel_size = trial_data[peyes.constants.PIXEL_SIZE_STR].values[0]
viewer_distance = trial_data[peyes.constants.VIEWER_DISTANCE_STR].values[0]
t = trial_data[peyes.constants.T].values
x = trial_data[peyes.constants.X].values
y = trial_data[peyes.constants.Y].values

x[(x < 0) | (x >= resolution[0])] = np.nan
y[(y < 0) | (y >= resolution[1])] = np.nan


**(3)** Labels

In [11]:
labels_df = pd.read_pickle(os.path.join(PROCESSED_DATA_DIR, DATASET_NAME, peyes.constants.LABELS_STR + ".pkl"))
labels_df = labels_df.xs(1, level=peyes.constants.ITERATION_STR, axis=1)            # Only use first iteration
labels_df = labels_df.xs(trial_id, level=peyes.constants.TRIAL_ID_STR, axis=1)      # Only use the relevant trial
labels_df = labels_df.dropna(how="all", axis=0)                                     # Drop rows with all NaNs

labeler_names = u.sort_labelers(labels_df.columns.get_level_values(peyes.constants.LABELER_STR).unique())

### Sub-Plots
**Generate the figure's subplots separately**

#### TOP: Gaze Trajectory

In [12]:
top_fig = peyes.visualize.gaze_trajectory(
    x=x, y=y, resolution=resolution, title="Gaze Trajectory",
    bg_image=img, bg_image_format='rgb', bg_alpha=0.5, marker_alpha=1,
    t=t, colorscale='Jet'
)
top_fig.update_layout(
    title=None,
)

# change colorbar title size
top_fig.data[-1]['marker']['colorbar']['title']['font']['size'] = 12

top_fig.show()

#### MIDDLE: Gaze Pixels vs. Time

In [13]:
middle_fig = peyes.visualize.gaze_over_time(
    x=x, y=y, t=t, resolution=resolution, title="Gaze Over Time",
    v=peyes._utils.pixel_utils.calculate_velocities(x, y, t), v_measure='px/s'
)

middle_fig.update_layout(
    paper_bgcolor='rgba(0, 0, 0, 0)',
    plot_bgcolor='rgba(0, 0, 0, 0)',
    height=top_fig.layout.height // 2,
    width=top_fig.layout.width,
    title=None,
    legend=dict(
        font=dict(size=10),
        orientation='v',
        yanchor='top', y=1,
        xanchor='left', x=0,
    ),
    yaxis=dict(
        title=dict(text='gaze position (px)', standoff=0),
        showgrid=True, zeroline=True, showline=True, rangemode='tozero',
    ),
    yaxis2=dict(
        title=dict(text='gaze velocity (px/s)', standoff=0),
        showgrid=False, zeroline=False, showline=True, rangemode='tozero',
    ),
    xaxis=dict(
        showgrid=False, showline=True, zeroline=False,
        range=[-(t.max() - t.min()) / 200, t.max() + (t.max() - t.min()) / 200]
    ),
)

middle_fig.show()

#### BOTTOM: Detector Scarfplots

In [14]:
bottom_fig = peyes.visualize.scarfplot_comparison_figure(
    t,
    *[labels_df[labeler_name] for labeler_name in labeler_names],
    names=labeler_names,
)
bottom_fig.update_layout(
    paper_bgcolor='rgba(0, 0, 0, 0)',
    plot_bgcolor='rgba(0, 0, 0, 0)',
    height=top_fig.layout.height // 2,
    width=top_fig.layout.width,
    title=None,
)

bottom_fig.show()